In [6]:
import random
import numpy as np

In [7]:
class Environment:
    def __init__(self, size, goal_state, rewards):
        self.size = size
        self.goal_state = goal_state
        self.rewards = rewards
        self.actions = ['up', 'down', 'left', 'right']
        self.action_space = len(self.actions)
        self.current_state = 0

    def reset(self):
        self.current_state = 0
        return self.current_state

    def get_next_state(self, state, action):
        row, col = divmod(state, self.size[1])
        if action == 'up':
            row = max(row - 1, 0)
        elif action == 'down':
            row = min(row + 1, self.size[0] - 1)
        elif action == 'left':
            col = max(col - 1, 0)
        elif action == 'right':
            col = min(col + 1, self.size[1] - 1)
        return row * self.size[1] + col

    def get_reward(self, state):
        return self.rewards.get(state, -1)
    
    def step(self, action):
        next_state = self.get_next_state(self.current_state, action)
        reward = self.get_reward(next_state)
        done = self.is_done(next_state)
        self.current_state = next_state
        return next_state, reward, done, {}
    
    def is_done(self, state):
        return state == self.goal_state

In [8]:
class Parameters:
    def __init__(self, size, goal_state, rewards, gamma):
        self.size = size
        self.goal_state = goal_state
        self.rewards = rewards
        self.gamma = gamma

In [9]:
class DynaQAgent:
    def __init__(self, environment, gamma=0.9, alpha=0.1, epsilon=0.1, n=10):
        self.environment = environment
        self.gamma = gamma
        self.alpha = alpha
        self.epsilon = epsilon
        self.n = n
        self.q_values = np.zeros((np.prod(environment.size), environment.action_space))
        self.model = {}
        self.initialize_model()

    def initialize_model(self):
        for state in range(np.prod(self.environment.size)):
            for action in range(self.environment.action_space):
                self.model[(state, action)] = (0, state)  # Initialize with zero reward and same state

    def select_action(self, state):
        if np.random.rand() < self.epsilon:
            return np.random.choice(self.environment.action_space)
        else:
            return np.argmax(self.q_values[state])

    def update_model(self, state, action, reward, next_state):
        self.model[(state, action)] = (reward, next_state)

    def planning_step(self):
        for _ in range(self.n):
            state, action = random.choice(list(self.model.keys()))
            reward, next_state = self.model[(state, action)]
            self.q_values[state][action] += self.alpha * (reward + self.gamma * np.max(self.q_values[next_state]) - self.q_values[state][action])

    def dyna_q(self, episodes=100):
        for _ in range(episodes):
            state = self.environment.reset()
            while True:
                action = self.select_action(state)
                next_state, reward, done, _ = self.environment.step(self.environment.actions[action])
                self.q_values[state][action] += self.alpha * (reward + self.gamma * np.max(self.q_values[next_state]) - self.q_values[state][action])
                self.update_model(state, action, reward, next_state)
                self.planning_step()
                if done:
                    break
                state = next_state

    def find_best_path_for_goal(self, start_state):
        state = start_state
        path = [state]
        action_map = {0: 'up', 1: 'down', 2: 'left', 3: 'right'}
        while True:
            action_idx = np.argmax(self.q_values[state])
            action = action_map[action_idx]
            next_state, reward, done, _ = self.environment.step(action)
            path.append(next_state)
            if done:
                break
            state = next_state
        return path

In [10]:
param = Parameters((3, 3), 8, {8: 10, 3: -5}, 0.9)
environment = Environment(param.size, param.goal_state, param.rewards)
agent = DynaQAgent(environment, param.gamma)

agent.dyna_q()

print("State Values:")
print(agent.q_values.max(axis=1).reshape(param.size))
print("\nPolicy:")
for row in range(param.size[0]):
    for col in range(param.size[1]):
        state = row * param.size[1] + col
        if state == param.goal_state:
            print(" G ", end=" ")
        else:
            action_idx = np.argmax(agent.q_values[state])
            print(environment.actions[action_idx], end=" ")
    print()

start_state = 0
best_path = agent.find_best_path_for_goal(start_state)
print("\nBest Path from state 0 to goal:")
print(best_path)

State Values:
[[ 4.57999956  6.19999984  7.99999999]
 [ 6.1999443   7.99999373 10.        ]
 [ 0.57862161  6.19988552  0.        ]]

Policy:
right right down 
right right down 
up up  G  

Best Path from state 0 to goal:
[0, 8]
